In [2]:
import pandas as pd
import ast
import json
import numpy as np

In [527]:
#checker = pd.read_json(r"D:\UH_Madagascar\annotations.json")


In [501]:
# determining input and output locations for files

input_f = r"D:\UH_Madagascar\annotations.json"
output_f = "cleaned_validation.csv"

with open(input_f, "r", encoding = "utf-8") as f:
    data = json.load(f)


In [502]:
# Cleaning the json export from label studio

cleaned_data = []

for record in data:
    """Here naming label studio id"""
    record_id = record.get("id")
    data_field = record.get("data", {})


    if isinstance(data_field, dict):
        """And here the id I gave by myself"""
        external_id = data_field.get("id", None)
        original_text = data_field.get("long_description", "")
    else:
        external_id = None
        original_text = ""

    annotations = record.get("annotations", [])
    if not isinstance(annotations, list) or not annotations:
    """Even though there were no annotation, the row is still conserved in set"""
        cleaned_data.append([record_id, external_id, original_text, "", ""])
        continue

    for annotation in annotations:
        results = annotation.get("result", [])
        """If annotation was found, save it as a list element"""
        if not isinstance(results, list) or not results:
            cleaned_data.append([record_id, external_id, original_text, "", ""])
            continue

        for entity in results:
            """What was the label given by the annotator"""
            value = entity.get("value", {})
            if not isinstance(value, dict):
                continue

            """And how are the annotation spans"""
            place_name = value.get("text", "").strip()
            labels = value.get("labels", [])
            start = value.get("start", "")
            end = value.get("end", "")
    
            if not place_name and not labels:
                cleaned_data.append([record_id, external_id, original_text, "", ""])
            else:
                labels_str = ", ".join(labels)
                cleaned_data.append([record_id, external_id, original_text, place_name, start, end, labels_str])
            

cleaned_df = pd.DataFrame(cleaned_data, columns = ["record_id", "external_id", "long_description", "place_name", "start", "end", "labels"])



In [503]:
#Saving clean annotation set

cleaned_df.to_csv(output_f, index=False)


In [ ]:
# Creating new columns for manually retrieved things

cleaned_df["coordinates"] = ""
cleaned_df["GN_ID"] = ""
#cleaned_df["GN_layer"] = ""

cleaned_df.dtypes

In [269]:
# Import geocoded datasets
base = pd.read_json("spacy_explson_id_all_rows.json")
#base2 = pd.read_json("spacy_e_s_explson_id.json")
#pelias = pd.read_json('pelias_exploded_with_1o.json')
#nomnom = pd.read_json('nominatim_coords_explson_id.json')

In [270]:
base['place_names'] = base['place_names'].fillna(0)
base

,aiddata_id,aiddata_2_id,year,donor,donor_iso,donor_region,implementing_agency,financing_agency,crs_bi_multi,recipient,...,WB_GeoLocID,WB_GeoLocName,WB_Latitude,WB_Longitude,WB_Country,id,place_names,ent_start,ent_end,ent_label
0,57106484,20554281,2003,Japan,JP,Far East Asia,0,JICA,1,Madagascar,...,0,0,0,0,0,1,0,0,0,0
1,57103097,10374723,1995,European Communities (EC),0,Europe,0,EDF,1,Madagascar,...,0,0,0,0,0,2,0,0,0,0
2,57051787,37965681,2008,United States,US,North & Central America,"PUBLIC SECTOR (donor, recipient, other)",MCC,1,Madagascar,...,0,0,0,0,0,3,0,0,0,0
3,57053923,29709047,2006,United States,US,North & Central America,NGO in Donor Country,AID,1,Madagascar,...,0,0,0,0,0,4,0,0,0,0
4,57055442,17327719,2002,Germany,DE,Europe,0,Found,1,Madagascar,...,0,0,0,0,0,5,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15866,0,0,2018,0,0,0,0,0,1,0,...,0,0,0,0,0,14269,0,0,0,0
15867,0,0,2018,0,0,0,0,0,1,0,...,0,0,0,0,0,14270,0,0,0,0
15868,0,0,2018,0,0,0,0,0,1,0,...,0,0,0,0,0,14271,0,0,0,0
15869,0,0,2018,0,0,0,0,0,1,0,...,0,0,0,0,0,14272,0,0,0,0


In [239]:
pelias

,aiddata_id,aiddata_2_id,year,donor,donor_iso,donor_region,implementing_agency,financing_agency,crs_bi_multi,recipient,...,WB_Longitude,WB_Country,id,place_names,ent_start,ent_end,ent_label,coordinates_pel,p_lon,p_lat
0,56986129,27739963,2006,France,FR,Europe,Public sector,AFD,1,Madagascar,...,0,0,10,['Madagascar'],32,42,LOC,"[[-18.628414, 46.704055]]",-18.628414,46.704055
1,56955232,18646796,2002,European Communities (EC),0,Europe,0,CEC,1,Madagascar,...,0,0,24,['MADAGASCAR'],27,37,GPE,"[[-18.628414, 46.704055]]",-18.628414,46.704055
2,56972700,28299995,2006,Norway,NO,Europe,Fredskorpset,MFA,1,Madagascar,...,0,0,29,['North and South'],98,113,LOC,[],NaN,NaN
3,56972700,28299995,2006,Norway,NO,Europe,Fredskorpset,MFA,1,Madagascar,...,0,0,29,['Madagascar YWCA'],206,221,GPE,"[[-18.628414, 46.704055], [-18.146694, 49.3954...",-18.628414,46.704055
4,56972700,28299995,2006,Norway,NO,Europe,Fredskorpset,MFA,1,Madagascar,...,0,0,29,['Kenya YMCA'],223,233,GPE,[],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3916,0,0,2018,0,0,0,0,0,1,0,...,0,0,14241,['TrodT'],289,294,LOC,[],NaN,NaN
3917,0,0,2018,0,0,0,0,0,1,0,...,0,0,14241,['pr(nTe'],684,690,LOC,[],NaN,NaN
3918,0,0,2018,0,0,0,0,0,1,0,...,0,0,14241,['la France'],695,704,LOC,[],NaN,NaN
3919,0,0,2018,0,0,0,0,0,1,0,...,0,0,14241,['Niger'],923,928,LOC,[],NaN,NaN


In [97]:
# GeoNames queries were restricted, so working on them here (CHECK pythoncode "geoparser_for_excel_aiddata_geonames")

geonam2 = pd.read_json(r'gn_expl_coords_2_5k.json')
geonam3 = pd.read_json(r'gn_expl_coords_7k.json')
geonam4 = pd.read_json(r'gn_expl_coords_10k.json')
geonam5 = pd.read_json(r'gn_expl_coords_12k.json')
geonam1 = pd.read_json(r'gn_expl_coords_1k.json')
geonam1['coordinates_gn'].dtype

dtype('O')

In [102]:
# Separating coordinate pairs in independent columns instead list elements

def extract_lon(coord):
    return coord [0][0] if isinstance(coord, list) and len(coord) > 0 and isinstance(coord[0], list) and len(coord[0]) == 2 else None

def extract_lat(coord):
    return coord [0][1] if isinstance(coord, list) and len(coord) > 0 and isinstance(coord[0], list) and len(coord[0]) == 2 else None

In [103]:
# Extracting coordinates of geonames datasets

geonam2['g_lon'] = geonam2['coordinates_gn'].apply(extract_lon)
geonam2['g_lat'] = geonam2['coordinates_gn'].apply(extract_lat)
geonam3['g_lon'] = geonam3['coordinates_gn'].apply(extract_lon)
geonam3['g_lat'] = geonam3['coordinates_gn'].apply(extract_lat)
geonam4['g_lon'] = geonam4['coordinates_gn'].apply(extract_lon)
geonam4['g_lat'] = geonam4['coordinates_gn'].apply(extract_lat)
geonam5['g_lon'] = geonam5['coordinates_gn'].apply(extract_lon)
geonam5['g_lat'] = geonam5['coordinates_gn'].apply(extract_lat)
geonam1['g_lon'] = geonam1['coordinates_gn'].apply(extract_lon)
geonam1['g_lat'] = geonam1['coordinates_gn'].apply(extract_lat)

In [115]:
# Creating a combination from separate geonames sets
rows = [geonam2, geonam3, geonam4, geonam5, geonam1]
geonams = pd.concat([pd.DataFrame(table) for table in rows], ignore_index=True)
geonams['g_lat'].dtype

dtype('float64')

In [116]:
#geonams.fillna('', inplace = True)

In [118]:
geonams.to_csv(r'D:\UH_Madagascar\Data\geonams_lat_lon.csv', index = False)


In [160]:
# Separating coords for Pelias and Nominatim, as well!

pelias['p_lon'] = pelias['coordinates_pel'].apply(extract_lon)
pelias['p_lat'] = pelias['coordinates_pel'].apply(extract_lat)
nomnom['n_lon'] = nomnom['coordinates_nom'].apply(extract_lon)
nomnom['n_lat'] = nomnom['coordinates_nom'].apply(extract_lat)

In [125]:
nomnom.to_csv(r'D:\UH_Madagascar\Data\nominatim_lat_lon.csv', index = False)


In [126]:
pelias['p_lon'] = pelias['coordinates_pel'].apply(extract_lon)
pelias['p_lat'] = pelias['coordinates_pel'].apply(extract_lat)

In [161]:
pelias

,aiddata_id,aiddata_2_id,year,donor,donor_iso,donor_region,implementing_agency,financing_agency,crs_bi_multi,recipient,...,WB_Longitude,WB_Country,id,place_names,ent_start,ent_end,ent_label,coordinates_pel,p_lon,p_lat
0,56986129,27739963,2006,France,FR,Europe,Public sector,AFD,1,Madagascar,...,0,0,10,[Madagascar],32,42,LOC,"[[-18.628414, 46.704055]]",-18.628414,46.704055
1,56955232,18646796,2002,European Communities (EC),0,Europe,0,CEC,1,Madagascar,...,0,0,24,[MADAGASCAR],27,37,GPE,"[[-18.628414, 46.704055]]",-18.628414,46.704055
2,56972700,28299995,2006,Norway,NO,Europe,Fredskorpset,MFA,1,Madagascar,...,0,0,29,[North and South],98,113,LOC,[],NaN,NaN
3,56972700,28299995,2006,Norway,NO,Europe,Fredskorpset,MFA,1,Madagascar,...,0,0,29,[Madagascar YWCA],206,221,GPE,"[[-18.628414, 46.704055], [-18.146694, 49.3954...",-18.628414,46.704055
4,56972700,28299995,2006,Norway,NO,Europe,Fredskorpset,MFA,1,Madagascar,...,0,0,29,[Kenya YMCA],223,233,GPE,[],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3916,0,0,2018,0,0,0,0,0,1,0,...,0,0,14241,[TrodT],289,294,LOC,[],NaN,NaN
3917,0,0,2018,0,0,0,0,0,1,0,...,0,0,14241,[pr(nTe],684,690,LOC,[],NaN,NaN
3918,0,0,2018,0,0,0,0,0,1,0,...,0,0,14241,[la France],695,704,LOC,[],NaN,NaN
3919,0,0,2018,0,0,0,0,0,1,0,...,0,0,14241,[Niger],923,928,LOC,[],NaN,NaN


In [144]:
pelias.to_csv(r'D:\UH_Madagascar\Data\pelias_lat_lon.csv', index = False)


In [271]:
# Change datatypes 
pelias['place_names'] = pelias['place_names'].astype('string')
base['place_names'] = base['place_names'].astype('string')
nomnom['place_names'] = nomnom['place_names'].astype('string')
geonams['place_names'] = geonams['place_names'].astype('string')


In [261]:
pelias['id'].dtype

dtype('int64')

In [272]:
# Merge Pelias georeferenced set with original set
# Later merging results of each geocoder
base = base.merge(pelias, 
           on=['id', 'place_names'], 
           suffixes = ('', '_x'), how = 'left')

base = base[[c for c in base.columns if not c.endswith('_x')]] # Drop duplicates that formed due the merging process
#base = base.loc[:, ~base.columns.str.endswith(('_x', '_y'))]


In [273]:
base

,aiddata_id,aiddata_2_id,year,donor,donor_iso,donor_region,implementing_agency,financing_agency,crs_bi_multi,recipient,...,WB_Longitude,WB_Country,id,place_names,ent_start,ent_end,ent_label,coordinates_pel,p_lon,p_lat
0,57106484,20554281,2003,Japan,JP,Far East Asia,0,JICA,1,Madagascar,...,0,0,1,0,0,0,0,NaN,NaN,NaN
1,57103097,10374723,1995,European Communities (EC),0,Europe,0,EDF,1,Madagascar,...,0,0,2,0,0,0,0,NaN,NaN,NaN
2,57051787,37965681,2008,United States,US,North & Central America,"PUBLIC SECTOR (donor, recipient, other)",MCC,1,Madagascar,...,0,0,3,0,0,0,0,NaN,NaN,NaN
3,57053923,29709047,2006,United States,US,North & Central America,NGO in Donor Country,AID,1,Madagascar,...,0,0,4,0,0,0,0,NaN,NaN,NaN
4,57055442,17327719,2002,Germany,DE,Europe,0,Found,1,Madagascar,...,0,0,5,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16921,0,0,2018,0,0,0,0,0,1,0,...,0,0,14269,0,0,0,0,NaN,NaN,NaN
16922,0,0,2018,0,0,0,0,0,1,0,...,0,0,14270,0,0,0,0,NaN,NaN,NaN
16923,0,0,2018,0,0,0,0,0,1,0,...,0,0,14271,0,0,0,0,NaN,NaN,NaN
16924,0,0,2018,0,0,0,0,0,1,0,...,0,0,14272,0,0,0,0,NaN,NaN,NaN


In [275]:
base1 = base.drop_duplicates(subset=['id', 'place_names', 'ent_start'])
base1

,aiddata_id,aiddata_2_id,year,donor,donor_iso,donor_region,implementing_agency,financing_agency,crs_bi_multi,recipient,...,WB_Longitude,WB_Country,id,place_names,ent_start,ent_end,ent_label,coordinates_pel,p_lon,p_lat
0,57106484,20554281,2003,Japan,JP,Far East Asia,0,JICA,1,Madagascar,...,0,0,1,0,0,0,0,NaN,NaN,NaN
1,57103097,10374723,1995,European Communities (EC),0,Europe,0,EDF,1,Madagascar,...,0,0,2,0,0,0,0,NaN,NaN,NaN
2,57051787,37965681,2008,United States,US,North & Central America,"PUBLIC SECTOR (donor, recipient, other)",MCC,1,Madagascar,...,0,0,3,0,0,0,0,NaN,NaN,NaN
3,57053923,29709047,2006,United States,US,North & Central America,NGO in Donor Country,AID,1,Madagascar,...,0,0,4,0,0,0,0,NaN,NaN,NaN
4,57055442,17327719,2002,Germany,DE,Europe,0,Found,1,Madagascar,...,0,0,5,0,0,0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16921,0,0,2018,0,0,0,0,0,1,0,...,0,0,14269,0,0,0,0,NaN,NaN,NaN
16922,0,0,2018,0,0,0,0,0,1,0,...,0,0,14270,0,0,0,0,NaN,NaN,NaN
16923,0,0,2018,0,0,0,0,0,1,0,...,0,0,14271,0,0,0,0,NaN,NaN,NaN
16924,0,0,2018,0,0,0,0,0,1,0,...,0,0,14272,0,0,0,0,NaN,NaN,NaN


In [291]:
# Merge Nominatim georeferenced set with original set
base2 = base2.merge(nomnom, 
           on=['id', 'place_names'], 
           suffixes = ('', '_x'),
                 how = 'left')

base2 = base2[[c for c in base2.columns if not c.endswith('_x')]]


In [292]:
base2 = base2.drop_duplicates(subset=['id', 'place_names', 'ent_start'])
base2

,aiddata_id,aiddata_2_id,year,donor,donor_iso,donor_region,implementing_agency,financing_agency,crs_bi_multi,recipient,...,place_names,ent_start,ent_end,ent_label,coordinates_pel,p_lon,p_lat,coordinates_nom,n_lon,n_lat
0,57106484,20554281,2003,Japan,JP,Far East Asia,0,JICA,1,Madagascar,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,57103097,10374723,1995,European Communities (EC),0,Europe,0,EDF,1,Madagascar,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,57051787,37965681,2008,United States,US,North & Central America,"PUBLIC SECTOR (donor, recipient, other)",MCC,1,Madagascar,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,57053923,29709047,2006,United States,US,North & Central America,NGO in Donor Country,AID,1,Madagascar,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,57055442,17327719,2002,Germany,DE,Europe,0,Found,1,Madagascar,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16921,0,0,2018,0,0,0,0,0,1,0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
16922,0,0,2018,0,0,0,0,0,1,0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
16923,0,0,2018,0,0,0,0,0,1,0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
16924,0,0,2018,0,0,0,0,0,1,0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


In [293]:
# Merge GeoNames georeferenced set with original set
base3 = base3.merge(geonams, 
           on=['id', 'place_names'], 
           suffixes = ('', '_x'),
                   how = 'left')

base3 = base3[[c for c in base3.columns if not c.endswith('_x')]]


In [295]:
base3 = base3.drop_duplicates(subset=['id', 'place_names', 'ent_start'])
base3

,aiddata_id,aiddata_2_id,year,donor,donor_iso,donor_region,implementing_agency,financing_agency,crs_bi_multi,recipient,...,ent_label,coordinates_pel,p_lon,p_lat,coordinates_nom,n_lon,n_lat,coordinates_gn,g_lon,g_lat
0,57106484,20554281,2003,Japan,JP,Far East Asia,0,JICA,1,Madagascar,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,57103097,10374723,1995,European Communities (EC),0,Europe,0,EDF,1,Madagascar,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,57051787,37965681,2008,United States,US,North & Central America,"PUBLIC SECTOR (donor, recipient, other)",MCC,1,Madagascar,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,57053923,29709047,2006,United States,US,North & Central America,NGO in Donor Country,AID,1,Madagascar,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,57055442,17327719,2002,Germany,DE,Europe,0,Found,1,Madagascar,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,0,0,2018,0,0,0,0,0,1,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21576,0,0,2018,0,0,0,0,0,1,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21577,0,0,2018,0,0,0,0,0,1,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21578,0,0,2018,0,0,0,0,0,1,0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [297]:
base3 = base3.fillna(0)
base3

,aiddata_id,aiddata_2_id,year,donor,donor_iso,donor_region,implementing_agency,financing_agency,crs_bi_multi,recipient,...,ent_label,coordinates_pel,p_lon,p_lat,coordinates_nom,n_lon,n_lat,coordinates_gn,g_lon,g_lat
0,57106484,20554281,2003,Japan,JP,Far East Asia,0,JICA,1,Madagascar,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
1,57103097,10374723,1995,European Communities (EC),0,Europe,0,EDF,1,Madagascar,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
2,57051787,37965681,2008,United States,US,North & Central America,"PUBLIC SECTOR (donor, recipient, other)",MCC,1,Madagascar,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
3,57053923,29709047,2006,United States,US,North & Central America,NGO in Donor Country,AID,1,Madagascar,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
4,57055442,17327719,2002,Germany,DE,Europe,0,Found,1,Madagascar,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,0,0,2018,0,0,0,0,0,1,0,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
21576,0,0,2018,0,0,0,0,0,1,0,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
21577,0,0,2018,0,0,0,0,0,1,0,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
21578,0,0,2018,0,0,0,0,0,1,0,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0


In [298]:
base3.to_csv('testi.csv')

In [299]:
base3['place_names'].dtype

string[python]

In [300]:
base_clean = base3.copy()

In [301]:
base_clean['place_names'] = base_clean['place_names'].str[1:]
base_clean['place_names'] = base_clean['place_names'].str[:-1]

In [303]:
base_clean

,aiddata_id,aiddata_2_id,year,donor,donor_iso,donor_region,implementing_agency,financing_agency,crs_bi_multi,recipient,...,ent_label,coordinates_pel,p_lon,p_lat,coordinates_nom,n_lon,n_lat,coordinates_gn,g_lon,g_lat
0,57106484,20554281,2003,Japan,JP,Far East Asia,0,JICA,1,Madagascar,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
1,57103097,10374723,1995,European Communities (EC),0,Europe,0,EDF,1,Madagascar,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
2,57051787,37965681,2008,United States,US,North & Central America,"PUBLIC SECTOR (donor, recipient, other)",MCC,1,Madagascar,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
3,57053923,29709047,2006,United States,US,North & Central America,NGO in Donor Country,AID,1,Madagascar,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
4,57055442,17327719,2002,Germany,DE,Europe,0,Found,1,Madagascar,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21575,0,0,2018,0,0,0,0,0,1,0,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
21576,0,0,2018,0,0,0,0,0,1,0,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
21577,0,0,2018,0,0,0,0,0,1,0,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0
21578,0,0,2018,0,0,0,0,0,1,0,...,0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0


In [306]:
base_clean['place_names'] = base_clean['place_names'].str.strip("'")

In [307]:
base_clean.to_csv('testi.csv', index=False)

In [3]:
#check = pd.read_csv(r'D:\UH_Madagascar\Data\validation_data\cleaned_annotations_w_coords.csv', encoding='utf-8', sep=';')
# The cleaned annotation set from the first part of the code was manually observed and validated, 
# then moved to validation_data -folder with new name.

check = pd.read_csv(r'D:\UH_Madagascar\Data\validation_data\validation_pack_cleaned.csv')
check = check.drop(columns=['bbox_GN'])

#check.rename(columns = {'external_id' : 'id'}, inplace = True)
#check.drop(['Unnamed: 0'], axis=1, inplace = True)
#check['annotations'][12]
#check['annotations'][4]
#check['annot_names'] = check['annot_names'].replace({'-': ''})
#check['annot_names'] = check['annot_names'].str.split(', ')
check

,id,ent_start,ent_end,ent_label,place_names,p_lat,p_lon,g_lat,g_lon,n_lat,...,annot_names,start,end,labels,WOF_locality,GN_ID,lat,lon,comments,bbox_WoF
0,56,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,-,0,0,-,-,-,0.0,0.0,-,NaN
1,96,36,52,GPE,VegetableOilbulk,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0.0,0.0,0,NaN
2,96,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,-,0,0,-,-,-,0.0,0.0,-,NaN
3,145,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,-,0,0,-,-,-,0.0,0.0,-,NaN
4,186,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,MANJARY,76,83,LOC,-,-,0.0,0.0,typo?,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,14121,0,14,LOC,P185 - Bourses,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0.0,0.0,0,NaN
215,14121,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,Madagascar,17,27,LOC,-,1062947,-20.0,47.0,-,NaN
216,14145,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,-,0,0,-,-,-,0.0,0.0,-,NaN
217,14242,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,Madagascar,32,42,LOC,-,1062947,-20.0,47.0,-,NaN


In [9]:
# Retrieving BoundingBoxes using GeoNames ID

from geopy.geocoders import GeoNames
import requests

username = "username"

def retrieve_bbox(geoname_id):
    """
    """
    url = f"https://secure.geonames.org/getJSON?geonameId={geoname_id}&username={username}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if "bbox" in data:
            return data["bbox"]    
    return None

In [10]:
check['bbox_GN'] = check['GN_ID'].apply(retrieve_bbox)

In [8]:
# Monstrous Bounding Box normalization since there was a string instance "AccuracyLevel in BBox-nest". 
# Cleanes the 'ACCLev' information, could be simplified

def parse_bbox(bbox):
    if isinstance(bbox, dict):
        return {k: float(v) if v is not None else None for k,v in bbox.items() if k != 'accuracyLevel'} # Dropping the accuracylevel infomration

    if isinstance(bbox, str):
        try:
            if "'" in bbox:
                bbox = bbox.replace("'", '"')
            bbox_dict = json.loads(bbox)
            return {k: float(v) if v is not None else None for k, v in bbox_dict.items() if k != 'accuracyLevel'}
        except (json.JSONDecodeError, ValueError, TypeError):
            return None
            
    return None


In [9]:
check['bbox'] = check['bbox'].apply(parse_bbox)

In [10]:
check

,id,ent_start,ent_end,ent_label,place_names,p_lat,p_lon,g_lat,g_lon,n_lat,...,start,end,labels,WOF_locality,GN_ID,lat,lon,comments,bbox_WoF,bbox
0,56,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,-,-,-,0.0,0.0,-,NaN,None
1,96,36,52,GPE,VegetableOilbulk,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,0.0,0,NaN,None
2,96,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,-,-,-,0.0,0.0,-,NaN,None
3,145,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,-,-,-,0.0,0.0,-,NaN,None
4,186,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,76,83,LOC,-,-,0.0,0.0,typo?,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,14121,0,14,LOC,P185 - Bourses,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0.0,0.0,0,NaN,None
215,14121,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,17,27,LOC,-,1062947,-20.0,47.0,-,NaN,"{'east': 50.486374859, 'south': -25.604970374,..."
216,14145,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,0,0,-,-,-,0.0,0.0,-,NaN,None
217,14242,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,32,42,LOC,-,1062947,-20.0,47.0,-,NaN,"{'east': 50.486374859, 'south': -25.604970374,..."


In [34]:
check.to_csv(r'D:\UH_Madagascar\Data\validation_data\cleaned_annotations_w_coords_w_bbox.csv', index = False)

In [11]:
check['bbox'].apply(type)

0      <class 'NoneType'>
1      <class 'NoneType'>
2      <class 'NoneType'>
3      <class 'NoneType'>
4      <class 'NoneType'>
              ...        
387    <class 'NoneType'>
388    <class 'NoneType'>
389        <class 'dict'>
390    <class 'NoneType'>
391    <class 'NoneType'>
Name: bbox, Length: 392, dtype: object

In [14]:
check['bbox'][165]

In [13]:
# Overlay analysis to calculate points inside BBoxes

def is_point_in_bbox (lat, lon, bbox):
    if not isinstance(bbox, dict) or not bbox:
        return False

    if any(value is None for value in bbox.values()):
        return False

    lat = float(lat)
    lon = float(lon)
    
    return bbox.get("south", float('-inf')) <= lat <= bbox.get("north", float('inf')) and \
            bbox.get("west", float('-inf')) <= lon <= bbox.get("east", float('inf'))

In [14]:
# Calculating points inside bboxes with GeoNames bboxes

check['pel_point_inside'] = check.apply(lambda row : is_point_in_bbox(row["p_lat"], row["p_lon"], row["bbox"]), axis = 1)
check['gn_point_inside'] = check.apply(lambda row : is_point_in_bbox(row["g_lat"], row["g_lon"], row["bbox"]), axis = 1)
check['nom_point_inside'] = check.apply(lambda row : is_point_in_bbox(row["n_lat"], row["n_lon"], row["bbox"]), axis = 1)

In [17]:
# Calculating points inside bboxes with WoF bboxes

check['pel_point_inside_2'] = check.apply(lambda row : is_point_in_bbox(row["p_lat"], row["p_lon"], row["bbox_WoF"]), axis = 1)
check['gn_point_inside_2'] = check.apply(lambda row : is_point_in_bbox(row["g_lat"], row["g_lon"], row["bbox_WoF"]), axis = 1)
check['nom_point_inside_2'] = check.apply(lambda row : is_point_in_bbox(row["n_lat"], row["n_lon"], row["bbox_WoF"]), axis = 1)

In [19]:
# Dropping duplicates

#check = check.drop(columns=['pel_point_inside_2', 'gn_point_inside_2', 'nom_point_inside_2'])
#check

,id,ent_start,ent_end,ent_label,place_names,p_lat,p_lon,g_lat,g_lon,n_lat,...,WOF_locality,GN_ID,lat,lon,comments,bbox_WoF,bbox,pel_point_inside,gn_point_inside,nom_point_inside
0,56,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,-,-,0.0,0.0,-,NaN,None,False,False,False
1,96,36,52,GPE,VegetableOilbulk,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0,NaN,None,False,False,False
2,96,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,-,-,0.0,0.0,-,NaN,None,False,False,False
3,145,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,-,-,0.0,0.0,-,NaN,None,False,False,False
4,186,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,-,-,0.0,0.0,typo?,NaN,None,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,14121,0,14,LOC,P185 - Bourses,0.0,0.0,0.0,0.0,0.0,...,0,0,0.0,0.0,0,NaN,None,False,False,False
215,14121,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,-,1062947,-20.0,47.0,-,NaN,"{'east': 50.486374859, 'south': -25.604970374,...",False,False,False
216,14145,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,-,-,0.0,0.0,-,NaN,None,False,False,False
217,14242,0,0,0,0,0.0,0.0,0.0,0.0,0.0,...,-,1062947,-20.0,47.0,-,NaN,"{'east': 50.486374859, 'south': -25.604970374,...",False,False,False


In [543]:
check.to_csv(r'D:\UH_Madagascar\Data\validation_data\cleaned_annotations_w_bbox.csv', encoding='utf-8', index=False)

In [539]:
check = pd.read_csv(r'D:\UH_Madagascar\Data\validation_data\cleaned_annotations_w_bbox.csv', encoding='utf-8')

#check['annot_names'] = check['annot_names'].str[2:]
#check['annot_names'] = check['annot_names'].str[:-2]


0                
1                
2                
3        NAMORAMA
4         MANJARY
          ...    
164              
165    Madagascar
166    Madagascar
167              
168    Madagascar
Name: annot_names, Length: 169, dtype: object

In [38]:
check2 = pd.read_csv(r'D:\UH_Madagascar\Data\validation_data\cleaned_annotations_w_bbox_final.csv', encoding='utf-8')
#check2['annot_names'] = check2['annot_names'].fillna('-')
#check2['start'] = check2['start'].replace({'-' : 0})

# Convert annotation start and end values into integer
check2['start'] = check2['start'].astype('int')
#check2['end'] = check2['end'].replace({'-' : 0})
check2['end'] = check2['end'].astype('int')

check2['start']

0        0
1        0
2        0
3       57
4       76
      ... 
164      0
165    271
166     17
167      0
168     32
Name: start, Length: 169, dtype: int32

In [46]:
check2[['m_start', 'm_end']] = check2[['start', 'end']]


0        0
1        0
2        0
3       65
4       83
      ... 
164      0
165    281
166     27
167      0
168     42
Name: m_end, Length: 169, dtype: int32

In [60]:
# Changin annotation span data types
check2 = check2.astype({'m_start': 'int64', 'm_end' : 'int64'})
check2['m_start'].dtype

dtype('int64')

In [30]:
tester = pd.read_csv(r'D:\UH_Madagascar\Data\geocoded_by_IA\combination_spacy_geocoders_origdata_cleannames.csv', encoding='utf-8')


C:\Users\iidahava\AppData\Local\Temp\ipykernel_1456\658221984.py:1: DtypeWarning: Columns (3,4,5,6,7,9,10,12,13,14,21,23,25,26,27,28,34,46,47,52,53,65,66,67,69,70,76,79,80,81,84,87,94,103,105,106,107,111,130,131,135,139,141,145) have mixed types. Specify dtype option on import or set low_memory=False.
  tester = pd.read_csv(r'D:\UH_Madagascar\Data\geocoded_by_IA\combination_spacy_geocoders_origdata_cleannames.csv', encoding='utf-8')


In [68]:
filt_tester = tester[tester["id"].isin(check2["id"])]
filt_tester = filt_tester[['id', 'long_description', 'ent_start', 'ent_end', 'ent_label','place_names', 'p_lon', 'p_lat', 'g_lon', 'g_lat', 'n_lon', 'n_lat']]

#M_start and m_end were created just for merging - and used quit recklessly as it is not neccessarily unique number
filt_tester[['m_start', 'm_end']] = filt_tester[['ent_start', 'ent_end']]
filt_tester

,id,long_description,ent_start,ent_end,ent_label,place_names,p_lon,p_lat,g_lon,g_lat,n_lon,n_lat,m_start,m_end
61,56,Watershed development by promotion of rural se...,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0
105,96,Commodity Sponsor: CRS 4190 MTNS of VegetableO...,36,52,GPE,VegetableOilbulk,0.0,0.0,0.0,0.0,0.0,0.0,36,52
154,145,TC AGGREGATED ACTIVITIES,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0
196,186,PROJECT HYGIENIC - HEALTH AND EDUCATIONAL DEVE...,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0
239,229,USAID Title II Food Aid. Monetization (Direct ...,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15344,13829,Ce projet vise a mettre en place les condition...,362,440,LOC,Tchelle.http://www.afd.fr/base-projets/consult...,0.0,0.0,0.0,0.0,0.0,0.0,362,440
15713,14121,P185 - Bourses - Madagascar - Bourses universi...,0,14,LOC,P185 - Bourses,0.0,0.0,0.0,0.0,0.0,0.0,0,14
15714,14121,P185 - Bourses - Madagascar - Bourses universi...,84,97,LOC,Campus France,0.0,0.0,0.0,0.0,0.0,0.0,84,97
15738,14145,Education de base-Enseignement primaire,0,0,0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0,0


In [69]:
# This merge copies almost all information to joined rows instead "long_description" 
# which is introduced only on row that was indicated to belong to specific set before
# The long description can be transferred to all cells using ID

merge = check2.merge(filt_tester, on=["id", "m_start", "m_end"], how="outer")
rest_merge = filt_tester[filt_tester["id"].isin(check2["id"])]
rest_merge = rest_merge.merge(check2[["id"]], on="id", how="right")
#final = pd.concat([rest_merge, merge]).sort_values(["id", "start"]).reset_index(drop =True)
final = pd.concat([rest_merge, merge]).drop_duplicates().sort_values(["id"]).reset_index(drop=True)
#final = pd.concat([check2, merge]).sort_values("id").reset_index(drop =True)
#final['record_id'] = final['record_id'].fillna(0)
#final['record_id'] = final['record_id'].astype('int')


,id,long_description,ent_start,ent_end,ent_label,place_names,p_lon,p_lat,g_lon,g_lat,...,start,end,labels,WOF_locality,GN_ID,lat,lon,comments,bbox,long_description_y
0,56,Watershed development by promotion of rural se...,0.0,0.0,0,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56,NaN,0.0,0.0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,-,-,-,-,-,-,NaN,Watershed development by promotion of rural se...
2,96,Commodity Sponsor: CRS 4190 MTNS of VegetableO...,36.0,52.0,GPE,VegetableOilbulk,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,-,-,-,-,-,-,NaN,NaN
4,96,NaN,36.0,52.0,GPE,VegetableOilbulk,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Commodity Sponsor: CRS 4190 MTNS of VegetableO...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,14145,Education de base-Enseignement primaire,0.0,0.0,0,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,14145,NaN,0.0,0.0,0,NaN,0.0,0.0,0.0,0.0,...,0.0,0.0,-,-,-,-,-,-,NaN,Education de base-Enseignement primaire
389,14242,P209 - Aide Alimentaire - PAM - Madagascar - S...,0.0,0.0,0,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390,14242,NaN,0.0,0.0,0,NaN,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P209 - Aide Alimentaire - PAM - Madagascar - S...


In [77]:
# m_start and m_end where just for merging rows with matching annotations between computed and ground truthed sets so they are dropped out
#final = final.drop(columns=['m_start', 'm_end'])

final['start'] = final['start'].fillna(0)
final['start'] = final['start'].astype('int')

final['end'] = final['end'].fillna(0)
final['end'] = final['end'].astype('int')

final['ent_start'] = final['ent_start'].fillna(0)
final['ent_start'] = final['ent_start'].astype('int')

final['ent_end'] = final['ent_end'].fillna(0)
final['ent_end'] = final['ent_end'].astype('int')

final

,id,long_description,ent_start,ent_end,ent_label,place_names,p_lon,p_lat,g_lon,g_lat,...,start,end,labels,WOF_locality,GN_ID,lat,lon,comments,bbox,long_description_y
0,56,Watershed development by promotion of rural se...,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56,NaN,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0,0,-,-,-,-,-,-,NaN,Watershed development by promotion of rural se...
2,96,Commodity Sponsor: CRS 4190 MTNS of VegetableO...,36,52,GPE,VegetableOilbulk,0.0,0.0,0.0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,96,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,-,-,-,-,-,-,NaN,NaN
4,96,NaN,36,52,GPE,VegetableOilbulk,0.0,0.0,0.0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Commodity Sponsor: CRS 4190 MTNS of VegetableO...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,14145,Education de base-Enseignement primaire,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,14145,NaN,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0,0,-,-,-,-,-,-,NaN,Education de base-Enseignement primaire
389,14242,P209 - Aide Alimentaire - PAM - Madagascar - S...,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390,14242,NaN,0,0,0,NaN,0.0,0.0,0.0,0.0,...,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P209 - Aide Alimentaire - PAM - Madagascar - S...


In [79]:

final['record_id'] = final['record_id'].fillna(0)
final['record_id'] = final['record_id'].astype('int')
final['ent_start'] = final['ent_start'].fillna(0)
final['ent_start'] = final['ent_start'].astype('int')
final['ent_end'] = final['ent_end'].fillna(0)
final['ent_end'] = final['ent_end'].astype('int')
final['record_id'] = final['record_id'].fillna(0)
final['record_id'] = final['record_id'].astype('int')

In [44]:
final = pd.concat([check2, merge]).sort_values("id").reset_index(drop =True)



In [594]:
# Create a counter for duplicate rows per ID
check2['id'] = check2.groupby('id').cumcount()
 
# Pivot table so that duplicate values become separate columns
check_pivoted = check2.pivot(index='record_id', columns=['id'], values=['annot_names','bbox', 'GN_ID'])
 
# Rename columns to make them more readable
check_pivoted.columns = [f'{col}' for col in check_pivoted.columns]
check_pivoted
# Merge with the main dataframe
#df_merged = df1.merge(df2_pivoted, on='id', how='left')

,"('annot_names', 0)","('annot_names', 1)","('annot_names', 2)","('annot_names', 3)","('annot_names', 4)","('annot_names', 5)","('annot_names', 6)","('annot_names', 7)","('annot_names', 8)","('annot_names', 9)",...,"('GN_ID', 134)","('GN_ID', 135)","('GN_ID', 136)","('GN_ID', 137)","('GN_ID', 138)","('GN_ID', 139)","('GN_ID', 140)","('GN_ID', 141)","('GN_ID', 142)","('GN_ID', 143)"
record_id,,,,,,,,,,,,,,,,,,,,,
35,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
57,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
86,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,MANJARY,NaN,NaN,NAMORAMA,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
138,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-,NaN,NaN,NaN,NaN
9972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1062947,NaN,NaN,NaN
10224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1062947,NaN,NaN


In [80]:
# Name was changed after validation to "cleaned_annotations_w_bbox_final"

final.to_csv(r'D:\UH_Madagascar\Data\validation_data\test3.csv', encoding='utf-8', index=False)

In [87]:
b_cleaned = pd.read_csv(r'D:\UH_Madagascar\Data\validation_data\annotations_bboxes_with_matches_2b_cleaned.csv', encoding='utf-8', sep = ';')
b_cleaned

,id,long_description,ent_start,ent_end,ent_label,place_names,record_id,start,end,annot_names,...,n_lat,long_description_x,labels,WOF_locality,GN_ID,lat,lon,comments,bbox,long_description_y
0,56,Watershed development by promotion of rural se...,0,0,0,NaN,0,0,0,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56,NaN,0,0,0,NaN,35,0,0,-,...,0.0,Watershed development by promotion of rural se...,-,-,-,-,-,-,NaN,Watershed development by promotion of rural se...
2,96,Commodity Sponsor: CRS 4190 MTNS of VegetableO...,36,52,GPE,VegetableOilbulk,0,0,0,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,96,NaN,0,0,NaN,NaN,57,0,0,-,...,NaN,Commodity Sponsor: CRS 4190 MTNS of VegetableO...,-,-,-,-,-,-,NaN,NaN
4,96,NaN,36,52,GPE,VegetableOilbulk,0,0,0,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Commodity Sponsor: CRS 4190 MTNS of VegetableO...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,14145,Education de base-Enseignement primaire,0,0,0,NaN,0,0,0,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
388,14145,NaN,0,0,0,NaN,10248,0,0,-,...,0.0,Education de base-Enseignement primaire,-,-,-,-,-,-,NaN,Education de base-Enseignement primaire
389,14242,P209 - Aide Alimentaire - PAM - Madagascar - S...,0,0,0,NaN,0,0,0,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
390,14242,NaN,0,0,0,NaN,0,0,0,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P209 - Aide Alimentaire - PAM - Madagascar - S...


In [100]:

# Masking using value counts - finding repeated entity_start positions
ent_s_dups = b_cleaned['ent_start'].value_counts() > 1
ent_mask = b_cleaned['ent_start'].map(ent_s_dups)  # True if 'a' is repeated

# finding repeated ids
id_dups = b_cleaned['id'].value_counts() > 1
id_mask = b_cleaned['id'].map(id_dups)  # True if 'd' is repeated

# Combining masks
both_repeated = ent_mask & id_mask  

# Dropping rows where 'record_id' == 0 and 'long_description' is NaN, but only if id and entity start positions are repeated
cleaned = b_cleaned[~(both_repeated & ((b_cleaned['record_id'] == 0) & (b_cleaned['long_description'].isna())))]

cleaned.head()

,id,long_description,ent_start,ent_end,ent_label,place_names,record_id,start,end,annot_names,...,n_lat,long_description_x,labels,WOF_locality,GN_ID,lat,lon,comments,bbox,long_description_y
0,56,Watershed development by promotion of rural se...,0,0,0,NaN,0,0,0,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,56,NaN,0,0,0,NaN,35,0,0,-,...,0.0,Watershed development by promotion of rural se...,-,-,-,-,-,-,NaN,Watershed development by promotion of rural se...
2,96,Commodity Sponsor: CRS 4190 MTNS of VegetableO...,36,52,GPE,VegetableOilbulk,0,0,0,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,96,NaN,0,0,NaN,NaN,57,0,0,-,...,NaN,Commodity Sponsor: CRS 4190 MTNS of VegetableO...,-,-,-,-,-,-,NaN,NaN
5,145,TC AGGREGATED ACTIVITIES,0,0,0,NaN,0,0,0,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
# Name changed to "*\data\validation_data\annotations_bboxes_with_matches_CLEAN.csv"

cleaned.to_csv(r'D:\UH_Madagascar\Data\validation_data\test.csv', encoding='utf-8', index=False)